In [1]:
import pickle
import numpy as np
from tqdm import tqdm

### Load pre-processed data

In [2]:
data_path = "/data1/CLERMS/data/"

In [ ]:
filename = data_path + "ALL_GNPS_positive_processed_annotated.pickle"
with open(filename, 'rb') as file:
    spectrums = pickle.load(file)

In [ ]:
inchikeys_list = [s.get("inchikey") for s in spectrums]
inchikey14_array = np.array([x[:14] for x in inchikeys_list])
inchikey14_unique = list({x[:14] for x in inchikeys_list})
print(len(inchikey14_unique))

### For all compounds 80% Train 20% Test 

In [ ]:
total_len = len(inchikey14_unique)
train_len = int(total_len * 0.8)
test_len = total_len - train_len
inchikey_test_index = np.random.choice(list(range(total_len)),test_len,replace=False)
inchikey_train_index = np.delete(np.arange(total_len),inchikey_test_index)
inchikey_test = np.array(inchikey14_unique)[inchikey_test_index]
inchikey_train = np.array(inchikey14_unique)[inchikey_train_index]
print(train_len,test_len)

#### For train and test set, if one compound has more than one spectra, extract one of them to 2nd set, the rest form 1st set.

In [ ]:
spectrum_train_1_index = []
spectrum_train_2_index = []
for key in tqdm(inchikey_train):
    index = np.where(inchikey14_array==key)[0]
    if index.shape[0]==1:
        spectrum_train_1_index.append(index[0])
    else:
        index_2 = np.random.choice(index)
        index_1 = index[index!=index_2]
        spectrum_train_1_index += list(index_1)
        spectrum_train_2_index.append(index_2)

In [ ]:
spectrum_test_1_index = []
spectrum_test_2_index = []
for key in tqdm(inchikey_test):
    index = np.where(inchikey14_array==key)[0]
    index_2 = np.random.choice(index)
    index_1 = index[index!=index_2]
    spectrum_test_1_index += list(index_1)
    spectrum_test_2_index.append(index_2)

In [ ]:
spectrums_inchikey14 = []
for idx, s in enumerate(tqdm(spectrums)):
    s = s.set("inchikey14",inchikey14_array[idx])
    spectrums_inchikey14.append(s)

In [ ]:
spectrums_array = np.array(spectrums_inchikey14)
train_1 = list(spectrums_array[spectrum_train_1_index])
train_2 = list(spectrums_array[spectrum_train_2_index])
test_1 = list(spectrums_array[spectrum_test_1_index])
test_2 = list(spectrums_array[spectrum_test_2_index])

#### Save

In [ ]:
pickle.dump(train_1,open(data_path + "train_1.pickle","wb"))
pickle.dump(train_2,open(data_path +"train_2.pickle","wb"))
pickle.dump(test_1,open(data_path +"test_1.pickle","wb"))
pickle.dump(test_2,open(data_path +"test_2.pickle","wb"))